In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import glob
from src.walking_recording import WalkingRecording
from typing import Dict

In [2]:
def setup() ->  Dict:
    """
    Fonction de setup de l'application
    Permet de récupérer les données des membres du groupe

    Returns:
        Dict: Dictionnaire contenant les données des membres
    """
    file_path = 'data/processed'
    members_list = glob.glob(f"{file_path}/*")
    members_dict = {}

    for member in members_list:
        member_name = os.path.basename(member)
        members_dict[member_name] = {}
        file_list = glob.glob(f"{member}/*")
        for csv_file in file_list:
            basename = os.path.basename(csv_file)
            members_dict[member_name][basename] = WalkingRecording.from_csv(csv_file)

    return members_dict


In [3]:
members = setup()
list(members.keys())

['Felix', 'Antoine', 'Leo', 'Matthieu', 'Corentin', 'Serge']

In [4]:
def members_metrics_df(members):
    metrics = []
    for name, member in members.items():
        for file_name, recording in member.items():
            metrics.append({
                "name" : name,
                "recording" : file_name,
                "steps" : len(recording.get_steps()),
                "frequency_fft" : recording.get_frequency_from_fft(),
                "frequency_steps" : recording.get_frequency_from_counting_steps(),
                "acceleration_std" : recording.get_std("acceleration (g)")
            })
    return pd.DataFrame(metrics)
        

In [5]:
df = members_metrics_df(members)
df.head()

,name,recording,steps,frequency_fft,frequency_steps,acceleration_std
0,Felix,accelerometer7_data.csv,17,1.901902,1.884570,0.283989
1,Felix,accelerometer6_data.csv,16,1.003009,1.867995,0.248067
2,Felix,accelerometer10_data.csv,14,1.900000,1.589242,0.219037
3,Felix,accelerometer1_data.csv,12,1.001001,1.443570,0.245534
4,Felix,accelerometer4_data.csv,17,1.900000,1.916168,0.273579


In [6]:
fig = px.scatter(df, x="frequency_steps", y="acceleration_std", color="name", hover_data=["recording"])
fig.show()

In [7]:
df.to_csv("./data/metrics.csv")